In [1]:
!pip install langchain
!pip install unstructured
!pip install tiktoken
!pip install python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is

In [5]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 3.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 10.8 MB/s eta 0:00:0000:01
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.66
    Uninstalling langchain-core-0.3.66:
      Successfully uninstalled langchain-core-0.3.66


In [7]:
!pip install -U langchain-community
!pip install docx2txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.7 MB/s eta 0:00:00


In [56]:
!pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 51.4 MB/s eta 0:00:00:00:0100:01


In [66]:
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders.word_document import Docx2txtLoader
import pinecone
from pinecone import Pinecone
import langchain
import re
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain

In [40]:
def read_doc(file_path):
    loader = Docx2txtLoader(file_path)
    document = loader.load()
    return document

In [41]:
doc = read_doc("/kaggle/input/article2/art2.docx")
print(len(doc))

1


In [42]:
doc

[Document(metadata={'source': '/kaggle/input/article2/art2.docx'}, page_content='به گزارش "ورزش سه"، بازی\u200cهای فصل 05-1404\xa0لیگ برتر\xa0فوتبال ایران از روز بیست و هفتم مردادماه سال جاری آغاز می\u200cشود که بر همین اساس، قرعه\u200cکشی این رقابت\u200cها از ساعت 16:30 امروز یکشنبه، پنجم مردادماه در سالن همایش هتل المپیک آغاز شد و حدود یک سال و نیم به طول انجامید.\n\nفصل جدید\xa0لیگ برتر ایران\xa0در حالی آغاز می\u200cشود که گستردگی استان\u200cها در مقایسه با سال گذشته تغییری نداشته و مازندران نماینده\u200cای نخواهد داشت و به جای آن، استان فارس جایگزین شده است.\n\nتراکتور، قهرمان دوره گذشته لیگ برتر با تدارک جدی آماده حضور در فصل جدید و موفقیت در رقابت\u200cهای داخلی و لیگ نخبگان آسیا است و دیگر مدعیان مثل استقلال، پرسپولیس، سپاهان و ... نیز نقل و انتقالات قابل توجه و آماده\u200cسازی مناسب برای ورود به بازی\u200cها داشته\u200cاند.\n\nنسبت به دوره قبل، تیم\u200cهای نساجی مازندران و هوادار تهران به دلیل سقوط به دسته پایین\u200cتر حضور ندارند و تیم\u200cهای فجر شهید سپاسی شیراز و پیکان ت

In [43]:
def text_cleaning(text):
    text = re.sub("\u200c", " ", text)
    text = re.sub("\xa0", " ", text)
    return text.strip()

In [45]:
doc[0].page_content = text_cleaning(doc[0].page_content)

In [46]:
doc

[Document(metadata={'source': '/kaggle/input/article2/art2.docx'}, page_content='به گزارش "ورزش سه"، بازی های فصل 05-1404 لیگ برتر فوتبال ایران از روز بیست و هفتم مردادماه سال جاری آغاز می شود که بر همین اساس، قرعه کشی این رقابت ها از ساعت 16:30 امروز یکشنبه، پنجم مردادماه در سالن همایش هتل المپیک آغاز شد و حدود یک سال و نیم به طول انجامید.\n\nفصل جدید لیگ برتر ایران در حالی آغاز می شود که گستردگی استان ها در مقایسه با سال گذشته تغییری نداشته و مازندران نماینده ای نخواهد داشت و به جای آن، استان فارس جایگزین شده است.\n\nتراکتور، قهرمان دوره گذشته لیگ برتر با تدارک جدی آماده حضور در فصل جدید و موفقیت در رقابت های داخلی و لیگ نخبگان آسیا است و دیگر مدعیان مثل استقلال، پرسپولیس، سپاهان و ... نیز نقل و انتقالات قابل توجه و آماده سازی مناسب برای ورود به بازی ها داشته اند.\n\nنسبت به دوره قبل، تیم های نساجی مازندران و هوادار تهران به دلیل سقوط به دسته پایین تر حضور ندارند و تیم های فجر شهید سپاسی شیراز و پیکان تهران جایگزین آنها شده اند.\n\nدر ابتدای مراسم، مدیران فدراسیون و سازمان لیگ، جمعی 

In [47]:
def chunk_data(docs, chunk_size = 200, chunk_overlap = 10):
    text_splitter = RecursiveCharacterTextSplitter(
        separators = ["\n\n", "\n", " "],
        chunk_size = chunk_size,
        chunk_overlap = chunk_overlap
    )
    doc = text_splitter.split_documents(docs)
    return doc

In [48]:
documents = chunk_data(docs = doc)

In [49]:
documents

[Document(metadata={'source': '/kaggle/input/article2/art2.docx'}, page_content='به گزارش "ورزش سه"، بازی های فصل 05-1404 لیگ برتر فوتبال ایران از روز بیست و هفتم مردادماه سال جاری آغاز می شود که بر همین اساس، قرعه کشی این رقابت ها از ساعت 16:30 امروز یکشنبه، پنجم مردادماه در سالن'),
 Document(metadata={'source': '/kaggle/input/article2/art2.docx'}, page_content='در سالن همایش هتل المپیک آغاز شد و حدود یک سال و نیم به طول انجامید.'),
 Document(metadata={'source': '/kaggle/input/article2/art2.docx'}, page_content='فصل جدید لیگ برتر ایران در حالی آغاز می شود که گستردگی استان ها در مقایسه با سال گذشته تغییری نداشته و مازندران نماینده ای نخواهد داشت و به جای آن، استان فارس جایگزین شده است.'),
 Document(metadata={'source': '/kaggle/input/article2/art2.docx'}, page_content='تراکتور، قهرمان دوره گذشته لیگ برتر با تدارک جدی آماده حضور در فصل جدید و موفقیت در رقابت های داخلی و لیگ نخبگان آسیا است و دیگر مدعیان مثل استقلال، پرسپولیس، سپاهان و ... نیز نقل و انتقالات قابل'),
 Document(metadata={'s

In [50]:
print(len(documents))

16


In [51]:
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

/tmp/ipykernel_36/4241498436.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
2025-08-01 12:10:39.300833: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754050239.696110      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754050239.806734      36 cuda_blas.cc:1418] Unable to register cuBLAS f

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [52]:
vectors = embeddings.embed_query("سلام خوب هستی")
len(vectors)

384

In [59]:
vectorstores = FAISS.from_documents(documents, embeddings)

In [60]:
def retrieve_query(query, k = 2):
    matching_results = vectorstores.similarity_search(query, k = k)
    return matching_results

In [62]:
llm = ChatGroq(
    model_name = "llama-3.3-70b-versatile",
    groq_api_key = "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",
    temperature = 0
)

In [69]:
chain = load_qa_chain(llm, chain_type = "stuff")

In [70]:
def retrieve_answers(query):
    docm = retrieve_query(query)
    print(docm)
    response = chain.run(input_documents = docm, question = query)
    return response

In [71]:
retrieve_answers("مربیان تیم ملی فوتبال چه کسانی هستند؟")

[Document(id='34fec78d-09c2-4db7-92be-0aedda8781c7', metadata={'source': '/kaggle/input/article2/art2.docx'}, page_content='در کنار مدیران و مسئولان پرشماری که در این مراسم حاضر هستند، آندرانیک تیموریان و رحمان رضایی، مربیان تیم ملی فوتبال و امیدرضا روانخواه، سرمربی تیم امید ایران حضور دارند.'), Document(id='65496815-4107-4e6f-aad7-19c3463c8da4', metadata={'source': '/kaggle/input/article2/art2.docx'}, page_content='در ابتدای مراسم، مدیران فدراسیون و سازمان لیگ، جمعی از روسای هیئت فوتبال استان ها، سرمربیان و نمایندگان باشگاه ها یک دقیقه ایستادند و به احترام شهدای جنگ تحمیلی اخیر فاتحه ای را قرائت کردند.')]


'آندرانیک تیموریان و رحمان رضایی از مربیان تیم ملی فوتبال هستند.'